# Extensibility

In [9]:
import datetime
from dataclasses import dataclass
from typing import Union, Set, Dict, List


```
Consider a restaurant chain that wants a notification system to help suppliers respond to demand:
* A restaurant may have a special, or be out of a certain ingredient, or indicate that some ingredient has gone bad. 
* In each case, the restaurant wants the supplier to automatically be notified that a restock is needed. 
* The supplier provides a Python library to do the actual notifications
```

In [2]:
# An implementation of the last library may be as follows

class Dish:
    pass

def out_of_stock(ingred):
    return

def get_items_in_stock():
    return

def declare_special(
    dish: Dish, 
    start_date: datetime.datetime,
    end_time: datetime.datetime
):
    # ... snip setup in local system ...
    # ... snip send notification to the supplier ...
    return

def order_dish(dish: Dish):
    # ... snip automated preparation
    out_of_stock_ingredients = {
        ingred 
        for ingred in dish
        if out_of_stock(ingred)
    }
    if out_of_stock_ingredients:
        # ... snip taking dishes off menu ...
        # ... snip send notification to the supplier ...
        # called every 24 hours
        return

def check_for_expired_ingredients():
    expired_ingredients = {
        ing 
        for ing in get_items_in_stock()
    }
    if expired_ingredients:
        # ... snip taking dishes off menu ...
        # ... snip send notifications to the supplier ...
        return


In [3]:
# But what if we need to send notifications to an email address of the company? 
# At first, we make the declare_special function take an email address:

class NotificationType:
    pass

class Email:
    pass

def declare_special(
    notification: NotificationType,
    start_date: datetime.datetime,
    end_time: datetime.datetime,
    email: Email
):
    # ... snip ...
    return


## Redesign for extensibility

In [4]:
# If we are not careful, we'll end up with a declare_special function like the following:

class PhoneNumber:
    pass

def declare_special(
    notification: NotificationType,
    start_date: datetime.datetime,
    end_time: datetime.datetime,
    emails: list[Email],
    texts: list[PhoneNumber],
    send_to_customer: bool
):
    # ... snip ...
    return


In [5]:
# Let's start by adding the notification types:

class Ingredient:
    pass

@dataclass
class NewSpecial:
    dish: Dish
    start_date: datetime.datetime
    end_date: datetime.datetime

@dataclass
class IngredientsOutOfStock:
    ingredients: Set[Ingredient]

@dataclass
class IngredientsExpired:
    ingredients: Set[Ingredient]

@dataclass
class NewMenuItem:
    dish: Dish

Notification = Union[NewSpecial, IngredientsOutOfStock, IngredientsExpired, NewMenuItem]


In [7]:
# The declare_special function should look like this:

def send_notification(notification: Notification):
    return 

def declare_special(
    dish: Dish, 
    start_date: datetime.datetime,
    end_date: datetime.datetime
):
    # ... snip setup in local system ...
    send_notification(NewSpecial(dish, start_date, end_date))



In [8]:
# Then we need to add the notification methods, as follows:

@dataclass
class Text:
    phone_number: str

@dataclass
class Email:
    email_address: str

@dataclass
class SupplierAPI:
    pass

NotificationMethod = Union[Text, Email, SupplierAPI]


In [ ]:
# And then we need to actually send a different notification type per method. 
# with helper functions, as follows:

def notify(notification_method: NotificationMethod, notification: Notification):
    if isinstance(notification_method, Text):
        send_text(notification_method, notification)
    elif isinstance(notification_method, Email):
        send_email(notification_method, notification)
    elif isinstance(notification_method, SupplierAPI):
        send_to_supplier(notification)
    else:
        raise ValueError("Unsupported Notification Method")

def send_text(text: Text, notification: Notification):
    if isinstance(notification, NewSpecial):
        # ... snip send text ...
        pass
    elif isinstance(notification, IngredientsOutOfStock):
        # ... snip send text ...
        pass
    elif isinstance(notification, IngredientsExpired):
        # ... snip send text ...
        pass
    elif isinstance(notification, NewMenuItem):
        # .. snip send text ...
        pass
    raise NotImplementedError("Unsupported Notification Method")

def send_email(email: Email, notification: Notification):
    # .. similar to send_text ...
    return

def send_to_supplier(notification: Notification):
    # .. similar to send_text
    return


In [10]:
# Now we have to tie it all together so we can add new users easily, as follows

users_to_notify: Dict[type, List[NotificationMethod]] = {
    NewSpecial: [
        SupplierAPI(), 
        Email("boss@company.org"),
        Email("marketing@company.org"), 
        Text("555-2345")
    ],
    IngredientsOutOfStock: [
        SupplierAPI(), 
        Email("boss@company.org")
    ],
    IngredientsExpired: [
        SupplierAPI(), 
        Email("boss@company.org")
    ],
    NewMenuItem: [
        Email("boss@company.org"), 
        Email("marketing@company.org")
    ],
}

def send_notification(notification: Notification):
    try:
        users = users_to_notify[type(notification)]
    except KeyError:
        raise ValueError("Unsupported Notification Method")
    
    for notification_method in users:
        notify(notification_method, notification)
